# **Assignment: Natural Language Query Processing**
This notebook demonstrates how to set up and run a natural language query processing application in Google Colab. The application extracts information from user queries, such as company names, metrics, and date ranges, and returns the data in JSON format.

## Step 1: Install Required Dependencies
This cell installs the necessary Python libraries for the application:
- **spaCy**: A popular natural language processing (NLP) library.
- **datefinder**: A library used to extract dates from strings in natural language text.
- **python-dateutil**: Provides extensions to the standard Python datetime module.

Run the following cell to install these dependencies.


In [ ]:
!pip install spacy datefinder python-dateutil

# Download and load spaCy model
import spacy
!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')


In [ ]:
import datefinder
from datetime import datetime, timedelta
import json
import re
from collections import deque
from dateutil.relativedelta import relativedelta

# Start with an empty list of known companies
known_companies = set()
known_metrics = {"gmv", "revenue", "profit"}

# List of common non-company words and months to filter out
non_company_words = {"give", "compare", "now", "add", "that", "with", "also"}
month_names = {"january", "february", "march", "april", "may", "june",
               "july", "august", "september", "october", "november", "december"}

# A deque to store the last four queries
query_history = deque(maxlen=4)

def standardize_metric_name(name):
    metric_variations = {
        "GMV": ["GMV", "gross merchandise volume"],
        "revenue": ["revenue", "sales"],
        "profit": ["profit", "income", "net income"]
    }
    name_lower = name.lower()
    for key, variations in metric_variations.items():
        if name_lower in [v.lower() for v in variations]:
            return key
    return None

def extract_companies_and_metrics(query):
    query_lower = query.lower()
    companies_in_query = {word.capitalize() for word in query_lower.split() if word in known_companies}
    metrics = [standardize_metric_name(word) for word in query_lower.split() if word in known_metrics]

    # Find and add any new company names based on capitalization (heuristic approach)
    potential_companies = [word.capitalize() for word in query.split() if word.istitle() and word.lower() not in non_company_words and word.lower() not in month_names]
    companies_in_query.update(potential_companies)

    # Add any newly identified companies to the known companies list
    known_companies.update(companies_in_query)

    return list(companies_in_query), metrics

def calculate_last_quarters(today, quarters_back=1):
    current_quarter = (today.month - 1) // 3 + 1
    target_quarter = current_quarter - quarters_back

    year = today.year - (1 if target_quarter <= 0 else 0)
    target_quarter = target_quarter + 4 if target_quarter <= 0 else target_quarter

    quarter_start_month = (target_quarter - 1) * 3 + 1
    start_date = datetime(year, quarter_start_month, 1).date()
    end_date = (start_date + relativedelta(months=3 * quarters_back)) - timedelta(days=1)

    return start_date, end_date

def parse_relative_date(expression, today):
    match = re.search(r"(yesterday|today|last|second last|from)\s*(\d*)\s*(year|month|week|day|quarter)?", expression, re.IGNORECASE)
    if not match:
        return today, today

    relative_type, number, unit = match.groups()
    number = int(number) if number else 1

    # Map relative units to their corresponding functions
    delta_functions = {
        "year": lambda n: (today - relativedelta(years=n), today),
        "month": lambda n: (today - relativedelta(months=n), today),
        "week": lambda n: (today - timedelta(weeks=n), today - timedelta(days=today.weekday() + 1)),
        "day": lambda n: (today - timedelta(days=n), today),
        "quarter": lambda n: calculate_last_quarters(today, n),
    }

    return delta_functions.get(unit or "day")(number)

def handle_special_cases(query, today):
    # Define special cases and their respective date adjustments
    special_cases = {
        "last 3 years": lambda: (today - relativedelta(years=3), today),
        "last 2 months": lambda: (today - relativedelta(months=2), today),
        "last quarter": lambda: calculate_last_quarters(today, 1),
        "last week": lambda: (today - timedelta(days=today.weekday() + 8), today - timedelta(days=today.weekday() + 1)),
        "yesterday": lambda: (today - timedelta(days=1), today - timedelta(days=1)),
        "last 6 months": lambda: (today - relativedelta(months=6), today),
        "last month": lambda: (today - relativedelta(months=1), today),
    }

    # Check if any special case applies to the query
    for case, func in special_cases.items():
        if case in query.lower():
            return func()

    # Default case: return today as both start and end date
    return today, today

def extract_info_from_query(query, previous_query={}):
    today = datetime.today().date()

    companies, metrics = extract_companies_and_metrics(query)

    # Use previous companies and metrics if context indicates a comparison or specific time period
    if "compare" in query or "with" in query or "add" in query or "last" in query:
        if not companies:
            companies = previous_query.get("companies", [])
        else:
            companies = list(set(companies + previous_query.get("companies", [])))
        if not metrics:
            metrics = previous_query.get("metrics", ["GMV"])

    # Extract dates
    dates = list(datefinder.find_dates(query))
    if dates:
        start_date = dates[0].date()
        end_date = dates[-1].date() if len(dates) > 1 else today
    else:
        # Handle special cases or default to previous query dates
        start_date, end_date = handle_special_cases(query, today)
        if not start_date or not end_date:
            start_date_str = previous_query.get("start_date", (today - relativedelta(years=5)).strftime("%Y-%m-%d"))
            end_date_str = previous_query.get("end_date", today.strftime("%Y-%m-%d"))
            start_date = datetime.strptime(start_date_str, "%Y-%m-%d").date()
            end_date = datetime.strptime(end_date_str, "%Y-%m-%d").date()

    # Correct the start date if a specific range like "from January 2023" is mentioned
    if "from" in query.lower() and dates:
        start_date = datetime(dates[0].year, dates[0].month, 1).date()  # Start from the first day of the month

    # Handle "last week" specifically to ensure correct end date
    if "last week" in query.lower():
        last_sunday = today - timedelta(days=today.weekday() + 1)  # Last Sunday
        start_date = last_sunday - timedelta(days=6)  # Monday of the last week
        end_date = last_sunday

    start_date = start_date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")

    if not companies:
        return {"error": "No company name found in the query."}
    if not metrics:
        return {"error": "No performance metric found in the query."}

    extracted_info = [{"entity": company, "parameter": metrics[0], "startDate": start_date, "endDate": end_date} for company in companies]

    # Store current query details in history (only keep the last 4)
    query_history.append({
        "companies": companies,
        "metrics": metrics,
        "start_date": start_date,
        "end_date": end_date
    })

    # Return the extracted information
    return extracted_info

# Hardcoded queries for testing
queries = [
    "Give me GMV for Apple for last 3 years",
    "Compare the GMV of Flipkart and Amazon for last 2 months.",
    "Compare the GMV of Flipkart and Amazon from January 2023 to December 2023.",
    "Compare the GMV of Flipkart and Amazon from January 2023 to last week.",
    "Now include Google and compare for last 6 months"
]

# Process each query while remembering only the last 4 queries
previous_query = {}
for query in queries:
    if query_history:
        previous_query = query_history[-1]
    result = extract_info_from_query(query, previous_query)
    print(f"Query: {query}")
    print(json.dumps(result, indent=4))
    print("\n")
